In [ ]:
import pylab
import matplotlib
import numpy as np
import pandas as pd
%matplotlib inline

# Data cleaning


Pandas is super versatile and can deal with a wide range of data formats. Usually though, a degree of data cleaning is needed to make it useable for a given application.  Consider the following spectrometer files:

In [ ]:
dat=pd.read_excel("./RLM_07112022_fullantiangle18c6wBa.xlsx",engine='openpyxl')
dat

Theres a lot of badness here.  First, the bottom rows all seem to be trash. As is row 1.  We only want to read the first 600 or so rows which have the data in. Lets chop off all the garbage at the bottom.

In [ ]:
dat=pd.read_excel("./RLM_07112022_fullantiangle18c6wBa.xlsx",engine='openpyxl')[1:602]
dat

Now we've got to do something about the column names. Many pathologies are present here:
1) The column names span two different rows; in Pandas the column names are not properly represented

2) The actual name of the series is for some mysterious reason one column left of where the actual data for that series is.

3) Wavelength occurs a totally unnecessary number of times, the same data for each series

In [ ]:
#Lets take care of 2 first. We need to rename each column of data with the name of the one that is placed one to the left of it.  
#We'll also call the left-most column "Wavelength", and plan to kill all the other wavelength columns shortly


#For this purpose we need a renaming mapper, a python dictionary that accepts the old column name and returns the new column name:

renamemapper={}

for i in range(0,len(dat.columns)-1):
    renamemapper[dat.columns[i+1]]=dat.columns[i]
renamemapper[dat.columns[0]]="Wavelength"



In [ ]:
# The dictionary works like this - you query it with the old name and it gives you the new one

renamemapper['Unnamed: 1']


In [ ]:
# Lets now destroy all the unnecessary columns. Wavelength occurs every even numbered column, 
# so these are the ones that need to die, in the existing labelling scheme:

WhichOnesToKill=range(2,len(dat.columns),2)
ToKill=dat.columns[WhichOnesToKill]
print(ToKill)

In [ ]:
dat_trimmed=dat.drop(ToKill,axis=1)
dat_trimmed

In [ ]:
#Now we'll apply the rename mapper:
dat_renamed=dat_trimmed.rename(renamemapper,axis=1).astype(float)
dat_renamed

Getting there! Now, according to this column naming convention, the columns names really mean something. The format is:

"Name_anti[angle]"

We'd like to organize them into some data structure that reflects this organizing principle.  Maybe say, a dictionary where the key is the angle, and the value is the data series? Lets try it:

In [ ]:
# First thing we need to do is extract the actual angle from the column name string. 
# A typical column name is like this:
examplename=dat_renamed.columns[1]
print(examplename)

In [ ]:
# What can we do to get the number out?  Here are some options that don't work and one that does:
#   1. Go through the string, and detect we are at at a number vs a letter, and keep the numbers.
#      --> doesnt work because 18 and 6170 also appear
#   2. Keep only the last 2 characters
#      --> doesnt work because sometimes the angle is only one digit long
#   3. Find an instance of "anti" and keep everythig right
#      --> this works, as long as some inventive spectrometer inventor doesnt change the name convention

In [ ]:
# The "split" command will divide a string based on instances of some substring. Like this:
SomeString="a_walrus_b_walrus_c_walrus_d_walrus_e"
SomeString.split("_walrus_")

In [ ]:
# So we can do this to split out the angle:
examplename.split("anti")[-1]

In [ ]:
#But note the '' there, it is a string '85' whereas we want a number 85. Convert it to a float:
float(examplename.split("anti")[-1])

In [ ]:
#OK, so with that solved, lets fill up our new data structure, indexed by the angle:
AntiAngleDict={}
ColumnNames=dat_renamed.columns
for name in ColumnNames:
    if "anti" in name:
        angle = float(name.split("anti")[-1])
        AntiAngleDict[angle]=np.array(dat_renamed[name])

In [ ]:
# We can extract from this structure the response at each angle
AntiAngleDict[85]

In [ ]:
list(AntiAngleDict.keys())

# Plotting


In [ ]:
# Time for some plots. We want to plot each angle one by one. First get a sorted list of angles 
# that are in the data structure:
angles=np.sort(list(AntiAngleDict.keys()))
angles

In [ ]:
# Lets use a nice color map that is reddest at the max angle and bluest at the min angle.
# Python color maps are by default indexed from 0 to 1. So we'll make a dictionary saying what
#   color to use for each angle:
colors={}

# Jet is a "rainbow" type color map in python. 
jet =  pylab.get_cmap('jet') 

for angle in angles:
    colors[angle]=jet(angle/max(angles))

# This is the RGBK index to the color for that angle
colors[45]

In [ ]:
# Now lets make a plot:

pylab.figure(figsize=(5,5),dpi=200)

for angle in angles:
    pylab.plot(dat_renamed.Wavelength, AntiAngleDict[angle],color=colors[angle],label=round(angle))

pylab.legend(loc='upper right') 
pylab.xlabel("Wavelength (nm)")
pylab.ylabel("Fluorescene Intensity (arb)")
pylab.ylim(0,50)

In [ ]:
# To use our scattering normalization trick, we need to know which rows 
# have wavelength in the interesting range. Thats these:

normframe=dat_renamed.loc[(200<dat_renamed.Wavelength) & (400>dat_renamed.Wavelength)]

normframe

In [ ]:
# Summing each column gives the normalization constant:

normsums= normframe.sum()
normsums

In [ ]:
# We need to do the angle thing again:
NormalizationDict={}
ColumnNames=dat_renamed.columns
for name in ColumnNames:
    if "anti" in name:
        angle = float(name.split("anti")[-1])
        NormalizationDict[angle]=float(normsums[name])

In [ ]:
# Now we can make the normalized plot:

pylab.figure(figsize=(5,5),dpi=200)

for angle in angles:
    pylab.plot(dat_renamed.Wavelength, AntiAngleDict[angle]/NormalizationDict[angle],color=colors[angle],label=round(angle))

pylab.legend(loc='upper right') 
pylab.xlabel("Wavelength (nm)")
pylab.ylabel("Fluorescene Intensity (arb)")
pylab.ylim(0,0.03)

# Condensing the hard work into portable functions


Once we've figured out the recipe, we can put all that into a function so we don't need a million rows of notebook every time we want a new plot. The following two functions condense down the process of reading the file and making the plot:

In [ ]:
def ReadFile(filename,toprange=602):

    AntiAngleDict={}  
    NormDict={}
    
    dat=pd.read_excel(filename,engine='openpyxl')[1:toprange]

    renamemapper={}
    for i in range(0,len(dat.columns)-1):
        renamemapper[dat.columns[i+1]]=dat.columns[i]
    renamemapper[dat.columns[0]]="Wavelength"
    dat= dat.rename(renamemapper,axis=1).astype(float)
    dat= dat.drop(dat.columns[range(2,len(dat.columns),2)],axis=1)
    
    dat_norm=dat.loc[(200<dat_renamed.Wavelength) & (400>dat_renamed.Wavelength)]
    
    for name in dat.columns:
        if("anti" in name):
            angle = float(name.split("anti")[-1])
            AntiAngleDict[angle]=np.array(dat[name])
            NormDict[angle]=float(dat_norm.sum()[name])

    return dat, AntiAngleDict, NormDict

In [ ]:
def MakePlot(dat, AntiAngleDict, NormDict):
    pylab.figure(figsize=(5,5),dpi=200)
    angles=np.sort(list(AntiAngleDict.keys()))
    for angle in angles:
        col=jet(angle/max(angles))
        pylab.plot(dat.Wavelength, AntiAngleDict[angle]/NormDict[angle],color=col,label=round(angle))
            
    pylab.legend(loc='upper right') 
    pylab.xlabel("Wavelength (nm)")
    pylab.ylabel("Fluorescene Intensity (arb)")
    pylab.ylim(0,0.05)
    pylab.show()

In [ ]:
# Read the file
dat_vaptes, aa_vaptes, nd_vaptes=ReadFile("./RLM_07112022_fullantiangleVAPTES.xlsx")
dat_18c6Ba, aa_18c6Ba, nd_18c6Ba=ReadFile("./RLM_07112022_fullantiangle18c6wBa.xlsx")

In [ ]:
# Make the original plot
MakePlot(dat_vaptes, aa_vaptes, nd_vaptes)
MakePlot(dat_18c6Ba, aa_18c6Ba, nd_18c6Ba)

In [ ]:
# Exercise:

# plot the scattering-normalized response of piranha glass, VAPTES, 18c6, 18c6+Ba at 
# anti35 degrees, using the files included in this github repo